In [9]:
import hashlib
import logging
from typing import Callable, List
from itertools import product
from multiprocessing import Pool, cpu_count
import sys

logging.basicConfig(
    filename='salt_checked.log',  # Имя лог-файла
    filemode='a',        # 'a' для добавления записей в файл, 'w' для перезаписи файла
    format='%(asctime)s - %(levelname)s - %(message)s',  # Формат логов
    level=logging.INFO   # Уровень логирования: DEBUG, INFO, WARNING, ERROR, CRITICAL
)

hash_functions = {
    'md5': hashlib.md5,
    'sha1': hashlib.sha1,
    'sha224': hashlib.sha224,
    'sha256': hashlib.sha256,
    'sha384': hashlib.sha384,
    'sha512': hashlib.sha512,
    'blake2b': hashlib.blake2b,
    'blake2s': hashlib.blake2s,
}

In [31]:
def salt_generator(min_length=int,max_length=int, type=int) -> list[str]:
    match type:
        case 0:
            charset = '0123456789'  # Только цифры
        case 1:
            charset = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'  # Только буквы
        case 2:
            charset = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'  # Цифры + буквы
        case _:
            raise ValueError("Недопустимый тип. Используйте 0 (цифры), 1 (буквы), или 2 (цифры и буквы).")
        
    out = []
    
    # Генерация всех возможных комбинаций от min_length до max_length
    for length in range(min_length, max_length + 1):
        out.extend(''.join(combo) for combo in product(charset, repeat=length))
    
    return out

In [32]:
def mega_salt_list(min_len: int, max_len: int) -> List[str]:
    out = []
    for i in range(min_len, max_len + 1):
        out += salt_generator(min_length=min_len, max_length=i, type=2)
    return out

In [30]:
def check_hash_function(input_data: str, output_hash: str, salt: str, hash_func):
    h = hash_func(input_data.encode()).hexdigest()
    if h == output_hash:
        logging.info(f'верная соль: \"{salt}\"')
        return True
    else:
        logging.info(f'неверная соль: \"{salt}\"')
        return False

In [33]:
def brute_force_salt_list(input_data, output_hash, salt_list, hash_func):
    for salt in salt_list:
        if check_hash_function(input_data,output_hash,salt,hash_func):
            return True
    return False

In [37]:
input_data = "89868999648"
output_hash = "00f162fae89b11e04f94016fc752ccef"
salt_list = mega_salt_list(min_len=1, max_len=3)


# Debugging statements
print(f"Generated {len(salt_list)} salts.")
print(f"First 10 salts: {salt_list[:10]}")
print(f"Last 10 salts: {salt_list[-10:]}")

Generated 246202 salts.
First 10 salts: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Last 10 salts: ['ZZQ', 'ZZR', 'ZZS', 'ZZT', 'ZZU', 'ZZV', 'ZZW', 'ZZX', 'ZZY', 'ZZZ']


In [38]:
brute_force_salt_list(input_data, output_hash, salt_list, hash_functions['md5'])

False

In [17]:
s = '123a'
s = s.encode('utf-8')
s

b'123a'

In [27]:
s_h = hashlib.md5(s).hexdigest()
sys.getsizeof(s_h) * 8

584

In [34]:
charset = '0123456789'
hex_needed = '48847b4b82f40447582e170d7b3b8e0e'
for combo in product(charset, repeat=11):
    number = (''.join(combo)).encode('utf-8')
    hex_num = hashlib.md5(number).hexdigest()
    if(hex_num == hex_needed):
        print(f'{number} -> {hex_num}')

KeyboardInterrupt: 